In [26]:
import jieba
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from tqdm import tqdm
from pprint import pprint
import os
import pandas as pd
import numpy as np

In [10]:
with open('chinese_stopwords.txt','r',encoding = 'utf-8') as fp:
    stopwords = [i[:-1] for i in fp.readlines()]

In [6]:
news = pd.read_csv('sqlResult.csv',encoding = 'gb18030')
print(news.shape)
news.head()

(89611, 7)


,id,author,source,content,feature,title,url
0,89617,NaN,快科技@http://www.kkj.cn/,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""37""...",小米MIUI 9首批机型曝光：共计15款,http://www.cnbeta.com/articles/tech/623597.htm
1,89616,NaN,快科技@http://www.kkj.cn/,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""15""...",骁龙835在Windows 10上的性能表现有望改善,http://www.cnbeta.com/articles/tech/623599.htm
2,89615,NaN,快科技@http://www.kkj.cn/,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,"{""type"":""科技"",""site"":""cnbeta"",""commentNum"":""18""...",一加手机5细节曝光：3300mAh、充半小时用1天,http://www.cnbeta.com/articles/tech/623601.htm
3,89614,NaN,新华社,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,"{""type"":""国际新闻"",""site"":""环球"",""commentNum"":""0"",""j...",葡森林火灾造成至少62人死亡 政府宣布进入紧急状态（组图）,http://world.huanqiu.com/hot/2017-06/10866126....
4,89613,胡淑丽_MN7479,深圳大件事,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,"{""type"":""新闻"",""site"":""网易热门"",""commentNum"":""978"",...",44岁女子约网友被拒暴雨中裸奔 交警为其披衣相随,http://news.163.com/17/0618/00/CN617P3Q0001875...


In [7]:
#处理缺失值
print(news[news.content.isna()].head(5))
news = news.dropna(subset  = ['content'])
print(news.shape)

         id author     source content  \
100   89517    NaN  中国证券报?中证网     NaN   
103   89514    NaN  中国证券报?中证网     NaN   
997   88620    NaN        央广网     NaN   
1273  88344    NaN        央广网     NaN   
1282  88335    NaN        央广网     NaN   

                                                feature  \
100   {"type":"公司","site":"中证网","commentNum":"0","jo...   
103   {"type":"公司","site":"中证网","commentNum":"0","jo...   
997   {"type":"时事要闻","site":"参考消息","commentNum":"0",...   
1273  {"type":"IT业界","site":"参考消息","commentNum":"0",...   
1282  {"type":"IT业界","site":"参考消息","commentNum":"0",...   

                                 title  \
100       天和防务股东未来6个月内计划减持不超过480万股公司股份   
103                    晶盛机电调整限制性股票回购价格   
997              [主播不在家]第二季：主播陈亮体验垃圾清运   
1273                LKK洛可可：想象力经济时代或已到来   
1282  CES2017：京东发布两款叮咚智能音箱新品 开放Alpha平台   

                                                    url  
100   http://www.cs.com.cn/ssgs/gsxw/201706/t2017062...  
103   http://www.cs.com.cn/ss

In [8]:
#分词
def split_text(text):
    text = text.replace(' ','')
    text = text.replace('\n','')
    text2 = jieba.cut(text.strip())
    result = ' '.join([w for w in text2 if w not in stopwords])
    return result

In [11]:
print(news.iloc[0].content)
print(split_text(news.iloc[0].content))

此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/体验版内测，稳定版暂不受影响），以确保工程师可以集中全部精力进行系统优化工作。有人猜测这也是将精力主要用到MIUI 9的研发之中。
MIUI 8去年5月发布，距今已有一年有余，也是时候更新换代了。
当然，关于MIUI 9的确切信息，我们还是等待官方消息。

 当然 MIUI9 确切 信息 等待 官方消息


In [12]:
if not os.path.exists("corpus.pkl"):
    #对所有文本进行分词
    corpus  = list(map(split_text,[str(i) for i in news.content]))
    print(corpus[0])
    print(len(corpus))
    print(corpus[0])
    
    #保存到文件，方便下次调用
    with open('corpus.pkl','wb') as file:
        pickle.dump(corpus,file)
else:
    #调用上次处理的结果
    with open('corpus.pkl','rb') as file:
        corpus = pickcle.load(file)

 当然 MIUI9 确切 信息 等待 官方消息
87054
 当然 MIUI9 确切 信息 等待 官方消息


In [13]:
#得到corpus的TF-IDF矩阵
countvectorizer = CountVectorizer(encoding = 'gb18030', min_df = 0.015)
tfidftransformer = TfidfTransformer()

In [16]:
tfidf = countvectorizer.fit_transform(corpus)
print(tfidf.shape)

(87054, 884)


In [19]:
label = list(map(lambda source: 1 if "新华" in str(source) else 0, news.source))

In [22]:
#数据集切分
X_train, X_test, y_train, y_test = train_test_split(tfidf.toarray(),label,test_size = 0.3, random_state = 42)
clf = MultinomialNB()
clf.fit(X = X_train,y = y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [24]:
y_predict = clf.predict(X_test)

def show_test_result(y_true,y_pred):
    print('accuracy:',accuracy_score(y_true,y_pred))
    print('precision:',precision_score(y_true,y_pred))
    print('recall:',recall_score(y_true,y_pred))
    print('f1_score:',f1_score(y_true,y_pred))
    
show_test_result(y_test,y_predict)

accuracy: 0.8666768771298388
precision: 0.9839923316558831
recall: 0.8670551965876937
f1_score: 0.9218301005747126


In [27]:
#使用模型检测抄袭新闻
prediction = clf.predict(tfidf.toarray())
labels = np.array(label)
compare_news_index = pd.DataFrame({'prediction':prediction,'labels':labels})
#预测是新华社新闻，但是实际上不是，则视为抄袭，得到这些新闻的index
copy_news_index = compare_news_index[(compare_news_index['prediction'] == 1)
                                     &(compare_news_index['labels'] == 0)].index 
#得到实际为新华社新闻的index
xinhuashe_news_index = compare_news_index[(compare_news_index['prediction'] ==1)].index
print('可能为copy的新闻条数：', len(copy_news_index))

可能为copy的新闻条数： 1094


In [33]:
if not os.path.exists("label.pkl"):
    # 使用k-means对文章进行聚类
    from sklearn.preprocessing import Normalizer
    from sklearn.cluster import KMeans
    normalizer = Normalizer()
    scaled_array = normalizer.fit_transform(tfidf.toarray())
    
    # 使用K-means，对全量文档进行聚类
    kmeans = KMeans(n_clusters = 25, random_state = 42, n_jobs = -1)
    k_labels = kmeans.fit_predict(scaled_array)
    
    #保存到文件，方便下次使用
    with open('label.pkl','wb') as file:
        pickle.dump(k_labels,file)
    print(k_labels.shape)
    print(k_labels[0])
    
else:
    #调用上次处理的结果
    with open('label.pkl','wb') as file:
        k_labels = pickle.load(file)
        
if not os.path.exists("id_class.pkl"):
    # 创建id_class
    id_class = {index:class_ for index, class_ in enumerate(k_labels)}
    # 保存到文件，方便下次调用
    with open('id_class.pkl','wb') as file:
        pickle.dump(id_class, file)
else:
    # 调用上次处理的结果
    with open('id_class.pkl','rb') as file:
        id_class = pickle.load(file)

if not os.path.exists("class_id.pkl"):
    from collections import defaultdict
    # 创建class_id字段，key为classId,value为文档index
    class_id = defaultdict(set)
    for index,class_ in id_class.items():
        # 只统计新华社发布的class_id
        if index in xinhuashe_news_index.tolist():
            class_id[class_].add(index)
    # 保存到文件，方便下次调用
    with open('class_id.pkl','wb') as file:
        pickle.dump(class_id, file)
else:
    # 调用上次处理的结果
    with open('class_id.pkl','rb') as file:
        class_id = pickle.load(file)

(87054,)
4


In [35]:
#输出每个类别的文档个数
count = 0
for k in class_id:
    print(count,len(class_id[k]))
    count+=1

0 6669
1 1983
2 4193
3 2163
4 1968
5 1535
6 1256
7 1409
8 1112
9 2324
10 1258
11 1993
12 437
13 1652
14 182
15 6079
16 1837
17 3127
18 11621
19 4438
20 3554
21 1782
22 1992
23 3713
24 1097


In [37]:
import editdistance
# 指定某篇文章的相似度
#print(copy_news_index)
cpindex = 3352 # 在copy_news_index
print('是否在新华社', cpindex in xinhuashe_news_index)
print('是否在copy_news', cpindex in copy_news_index)
print('3134是否在新华社', 3134 in xinhuashe_news_index)
print('3134是否在copy_news', 3134 in copy_news_index)

是否在新华社 False
是否在copy_news False
3134是否在新华社 False
3134是否在copy_news False


In [49]:
# 查找相似文本（使用聚类结果进行filter）
def find_similar_text(cpindex, top=10):
    # 只在新华社发布的文章中查找
    dist_dict={i:cosine_similarity(tfidf[cpindex],tfidf[i]) for i in class_id[id_class[cpindex]]}
    # 从大到小进行排序
    return sorted(dist_dict.items(),key=lambda x:x[1][0], reverse=True)[:top]

In [50]:
#print(cpindex)
similar_list = find_similar_text(cpindex)
print(similar_list)
print('怀疑抄袭:\n', news.iloc[cpindex].content)
# 找一篇相似的原文
similar2 = similar_list[0][0]
print('相似原文:\n', news.iloc[similar2].content)
# 求任意两篇文章的编辑距离 
print('编辑距离:',editdistance.eval(corpus[cpindex], corpus[similar2]))

[(29777, array([[0.82516542]])), (63689, array([[0.68450449]])), (61398, array([[0.40178646]])), (19115, array([[0.34947926]])), (56905, array([[0.28478808]])), (3000, array([[0.26553588]])), (22688, array([[0.23966775]])), (17674, array([[0.23796989]])), (17528, array([[0.18896121]])), (49521, array([[0.18148518]]))]
怀疑抄袭:
 　　中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨。5月份15个一线和热点二线城市房地产市场基本稳定，5月份房地产调控政策效果继续显现。
　　统计局：15个一线和热点二线城市房价同比涨幅全部回落
　　国家统计局城市司高级统计师刘建伟解读5月份房价数据
　　5月份一二线城市房价平均涨幅继续回落
　　国家统计局今日发布了2017年5月份70个大中城市住宅销售价格统计数据。对此，国家统计局城市司高级统计师刘建伟进行了解读。
　　一、15个一线和热点二线城市新建商品住宅价格同比涨幅全部回落、9个城市环比下降或持平
　　5月份，因地制宜、因城施策的房地产调控政策效果继续显现，15个一线和热点二线城市房地产市场基本稳定。从同比看，15个城市新建商品住宅价格涨幅均比上月回落，回落幅度在0.5至6.4个百分点之间。从环比看，9个城市新建商品住宅价格下降或持平；5个城市涨幅在0.5%以内。
　　二、70个大中城市中一二线城市房价同比涨幅持续回落
　　5月份，70个城市中新建商品住宅和二手住宅价格同比涨幅比上月回落的城市分别有29和18个。其中，一二线城市同比涨幅回落尤其明显。据测算，一线城市新建商品住宅和二手住宅价格同比涨幅均连续8个月回落，5月份比4月份分别回落2.2和1.7个百分点；二线城市新建商品住宅和二手住宅价格同比涨幅分别连续6个月和4个月回落，5月份比4月份分别回落0.8和0.5个百分点。
　　三、70个大中城市中房价环比下降及涨幅回落城市个数均有所增加
　　5月份，70个城市中新建商品住宅价格环比下降的城市有9

In [52]:
def find_similar_sentence(candidate,raw):
    similist = []
    cl = candidate.strip().split('。')
    ra = raw.strip().split('。')
    for c in cl:
        for r in ra:
            similist.append([c,r,editdistance.eval(c,r)])
    #最相似的5个句子
    sort = sorted(similist,key = lambda x:x[2])[:5]
    for c,r,ed in sort:
        if c != '' and r!='':
            print('怀疑抄袭句：{0}\n相似原句：{1}\n编辑距离：{2}\n'.format(c,r,ed))
#查找copy文章和第一相似的原文的对比
find_similar_sentence(news.iloc[cpindex].content,news.iloc[similar2].content)

怀疑抄袭句：其中，一二线城市同比涨幅回落尤其明显
相似原句：\n新华社记者陶亮摄
编辑距离：18

怀疑抄袭句：对此，国家统计局城市司高级统计师刘建伟进行了解读
相似原句：\n新华社记者陶亮摄
编辑距离：24

怀疑抄袭句：对此，国家统计局城市司高级统计师刘建伟进行了解读
相似原句：\n4月18日，国家统计局发布3月份70个大中城市住宅销售价格数据
编辑距离：27

怀疑抄袭句：中国5月份56座城市新建商品住宅价格环比上涨，4月份为58座上涨
相似原句：统计显示，3月份，70个城市中有8个城市新建商品住宅价格比上月下降，其他城市环比均有所上涨
编辑距离：29



AttributeError: 'str' object has no attribute 'ed'